# Gesture Based Tool for Sterile Browsing of Radiology Images

# Model Building

### Importing libraries

In [ ]:
# This library helps add support for large, multi-dimensional arrays and matrices
import numpy as np
#open source used for both ML and DL for computation
import tensorflow as tf
#it is a plain stack of layers
from tensorflow.keras.models import Sequential 
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten, Dropout
#Faltten-used fot flattening the input or change the dimension, MaxPooling2D-for downsampling the image for Convolutional layer
from tensorflow.keras.layers import Convolution2D,MaxPooling2D 
#Its used for different augmentation of the image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Augmenting the data 

In [ ]:
#setting parameter for Image Data agumentation to the traing data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [5]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(r'/content/drive/MyDrive/train',
                                            target_size=(64, 64),
                                            batch_size=3,
                                            color_mode='grayscale',
                                            class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(r'/content/drive/MyDrive/train',
                                          target_size=(64, 64),
                                          batch_size=3,
                                          color_mode='grayscale',
                                          class_mode='categorical') 

Found 594 images belonging to 6 classes.
Found 594 images belonging to 6 classes.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(x_train.class_indices)#checking the number of classes

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}


### Model Creation

In [ ]:
# Initializing the CNN
model = Sequential()

In [ ]:
# First convolution layer and pooling
model.add(Convolution2D(32, (3, 3), input_shape=(64, 64, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
# Second convolution layer and pooling
model.add(Convolution2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
# Flattening the layers i.e. input layer
model.add(Flatten())

In [ ]:
# Adding a fully connected layer, i.e. Hidden Layer
model.add(Dense(units=512 , activation='relu'))

In [ ]:
# softmax for categorical analysis, Output Layer
model.add(Dense(units=6, activation='softmax')) 

In [ ]:
model.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 512)               3

### Model Compilation

In [ ]:
# Compiling the CNN
# categorical_crossentropy for more than 2
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

## Model fitting

In [ ]:
# It will generate packets of train and test data for training
model.fit_generator(x_train,
                    steps_per_epoch = 594/3 , 
                    epochs = 25, 
                    validation_data = x_test,
                    validation_steps = 30/3 )

### Saving model

In [ ]:
# Save the model
model.save('Tested_gesture.h5')

In [ ]:
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)